In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import when

C:\Users\HP\python app\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spark=SparkSession.builder.appName('Random Forest Classifier').getOrCreate()
data=spark.read.csv(r'D:\priya\Ecommerce_Customers.csv',header=True,inferSchema=True)
data.printSchema()
data.show(3)
data=data.withColumn("churn",when(data['Yearly Amount Spent']<500,1).otherwise(0))
Assembler=VectorAssembler(
    inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
    outputCol='features'
)
output=Assembler.transform(data)
final_data=output.select('features','churn')
train_data,test_data=final_data.randomSplit([0.7,0.3])
rf=RandomForestClassifier(labelCol='churn',featuresCol='features')
rf_model=rf.fit(train_data)
pred_data=rf_model.transform(test_data)
evaluator=MulticlassClassificationEvaluator(labelCol='churn',predictionCol='prediction',metricName='accuracy')
accuracy=evaluator.evaluate(pred_data)
print("Random Forest Classifier accuracy:",accuracy)
pred_data.select("features","churn","prediction").show(5)
spark.stop()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Ar